In [1]:
import time 
start = time.time()

### Загрузка библиотек

In [2]:
import sys
sys.path.append('C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/!workspace/dict_ref_moduls/moduls')

import pandas as pd
import dataremake
import docx
from IPython.display import display
from numpy import nan

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

### Объявление функций

In [3]:
#  Функция подшивки числа категорий у категориальных атрибутов показателей
def category_count(row):
    temp_dict = {}
    for col in ['area_type',
                'age_group', 
                'year_of_birth', 
                'sex', 
                'month', 
                'birth_order',       
                'mother_marital_status', 
                'death_cause', 
                'bride_age', 
                'groom_age',       
                'migration_direction', 
                'country', 
                'region', 
                'territory_type',       
                'registration_type', 
                'citizenship',]:
        name = col
        count = len(list(data_origin[col].unique())) - 1
        temp_dict.update({col: count})
    
    attr = row['attributes']
    if attr == 'Нет':
        return 1
    else:
        name = data_origin_structure.loc[data_origin_structure.index[data_origin_structure['russian description'] == attr].tolist()[0]]['field name']
        return temp_dict[name]

In [4]:
# Функция подшивки максимального года к датасету индикаторов
def  max_year(row):
    name = row['indicator_name']
    max_year = indicators_data.loc[indicators_data.index[indicators_data['indicator'] == name].tolist()[0]]['max_year']
    return max_year

### Объявление констант

In [5]:
categorial_columns_list = [
                           'object_name',
                           'area_type',
                           'age_group',
                           'year_of_birth',
                           'sex',
                           'month',
                           'birth_order',
                           'mother_marital_status',
                           'death_cause',
                           'bride_age',
                           'groom_age',
                           'migration_direction',
                           'country',
                           'region',
                           'territory_type',
                           'registration_type',
                           'citizenship'
                          ]

In [6]:
# Путь к файлам базового датасета
data_origin_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/0_origin_data/'

# Путь к файлам разных словарей
dict_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/!workspace/dict_ref_moduls/'

### Загрузка датасетов

In [7]:
%%time

data_origin = pd.read_csv(data_origin_path + 'data-20210430-structure-20210430.csv', error_bad_lines=False, low_memory=False)
data_origin_meta = pd.read_csv(data_origin_path + 'meta.csv', encoding='latin-1')
data_origin_methodology = pd.read_csv(data_origin_path + 'methodology.csv')
data_origin_structure = pd.read_csv(data_origin_path + 'structure-20210430.csv')

<timed exec>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.




CPU times: total: 2min 18s
Wall time: 2min 52s


In [8]:
new_indicators_data = pd.read_csv(dict_path + 'new_indicators_data.csv', index_col='Unnamed: 0')

In [9]:
indicators_data = pd.read_csv(dict_path + 'indicators_data.csv', index_col='Unnamed: 0')

### Работа с датасетами

In [10]:
data_origin.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340687 entries, 0 to 9340686
Data columns (total 28 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   object_name            object 
 2   object_oktmo           float64
 3   object_okato           float64
 4   year                   int64  
 5   indicator_name         object 
 6   indicator_unit         object 
 7   original_unit          object 
 8   area_type              object 
 9   age_group              object 
 10  year_of_birth          float64
 11  sex                    object 
 12  month                  object 
 13  birth_order            object 
 14  mother_marital_status  object 
 15  death_cause            object 
 16  bride_age              object 
 17  groom_age              object 
 18  migration_direction    object 
 19  country                object 
 20  region                 object 
 21  territory_type         object 
 22  registration_type 

In [11]:
data_origin.year.min()

1990

In [12]:
ind_1 = 'Число умерших по причинам смерти'
ind_2 = 'Число детей, умерших в возрасте до 5 лет по причинам смерти'

# for u in list(data_origin[(data_origin.indicator_name == ind_1) | (data_origin.indicator_name == ind_2)].age_group.unique()):
#     print(u)

data_origin[(data_origin.indicator_name == ind_1) & (data_origin.year == 2019)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145152 entries, 3713399 to 5115707
Data columns (total 28 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     145152 non-null  int64  
 1   object_name            145152 non-null  object 
 2   object_oktmo           128520 non-null  float64
 3   object_okato           128520 non-null  float64
 4   year                   145152 non-null  int64  
 5   indicator_name         145152 non-null  object 
 6   indicator_unit         145152 non-null  object 
 7   original_unit          145152 non-null  object 
 8   area_type              145152 non-null  object 
 9   age_group              145152 non-null  object 
 10  year_of_birth          0 non-null       float64
 11  sex                    145152 non-null  object 
 12  month                  0 non-null       object 
 13  birth_order            0 non-null       object 
 14  mother_marital_status  0 non-

In [13]:
ind = 'Число детей, умерших в возрасте до 5 лет по причинам смерти'
print(ind.split(' — ')[0])
for col in categorial_columns_list:
    cond_1 = (data_origin['indicator_name'] == ind)
    if data_origin[cond_1][col].isna().sum()/len(data_origin[cond_1]) != 1:    
        print(col)
        print(data_origin[cond_1][col].nunique())
        print(data_origin[cond_1][col].unique().tolist())
print(data_origin[data_origin.indicator_name == ind].original_unit.unique())
print(data_origin[data_origin.indicator_name == ind].year.unique())
print()

Число детей, умерших в возрасте до 5 лет по причинам смерти
object_name
107
['Агинский Бурятский автономный округ', 'Алтайский край', 'Амурская область', 'Архангельская область', 'Архангельская область без Ненецкого автономного округа', 'Астраханская область', 'Белгородская область', 'Брянская область', 'Владимирская область', 'Волгоградская область', 'Вологодская область', 'Воронежская область', 'Дальневосточный федеральный округ', 'Еврейская автономная область', 'Забайкальский край', 'Ивановская область', 'Иркутская область', 'Кабардино-Балкарская Республика', 'Калининградская область', 'Калужская область', 'Камчатская область', 'Камчатский край', 'Карачаево-Черкесская Республика', 'Кемеровская область – Кузбасс', 'Кировская область', 'Коми-Пермяцкий автономный округ', 'Корякский автономный округ', 'Костромская область', 'Краснодарский край', 'Красноярский край', 'Курганская область', 'Курская область', 'Ленинградская область', 'Липецкая область', 'Магаданская область', 'Москва', 'Мо

In [14]:
data_origin.month.unique()

array([nan, 'январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль',
       'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',
       'месяц не указан', 'все месяца', 'все месяца, в т.ч. неуказанный'],
      dtype=object)

In [15]:
ind = 'Численность населения на 1 января'
cond_1 = (data_origin['indicator_name'] == ind)
cond_2 = (data_origin['object_name'] == 'Российская Федерация')
cond_3 = (data_origin['year'] == 2020)
cond_4 = (data_origin['area_type'] == 'городское и сельское население')
cond_5 = (data_origin['age_group'] == 'все возраста')
cond_6 = (data_origin['sex'] == 'мужчины и женщины')
cond_7 = (data_origin['migration_direction'] == 'мужчины и женщины')
cond_8 = (data_origin['country'] == 'Страны СНГ и Балтии')
cond_9 = (data_origin['year_of_birth'] == 1990.0)
data_origin[cond_1 & cond_2 & cond_9].head()

,id,object_name,object_oktmo,object_okato,year,indicator_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,indicator_value,original_accuracy,methodology,notes
32784,32785,Российская Федерация,NaN,NaN,1991,Численность населения на 1 января,человек,человек,городское и сельское население,до 1 года,1990.0,женщины,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,957048.0,0.0,population_1,пересчет от итогов ВПН-2002
35846,35847,Российская Федерация,NaN,NaN,1992,Численность населения на 1 января,человек,человек,городское и сельское население,1 год,1990.0,женщины,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,961147.0,0.0,population_1,пересчет от итогов ВПН-2002
38908,38909,Российская Федерация,NaN,NaN,1993,Численность населения на 1 января,человек,человек,городское и сельское население,2 года,1990.0,женщины,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,964736.0,0.0,population_1,пересчет от итогов ВПН-2002
41970,41971,Российская Федерация,NaN,NaN,1994,Численность населения на 1 января,человек,человек,городское и сельское население,3 года,1990.0,женщины,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,969245.0,0.0,population_1,пересчет от итогов ВПН-2002
45032,45033,Российская Федерация,NaN,NaN,1995,Численность населения на 1 января,человек,человек,городское и сельское население,4 года,1990.0,женщины,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,978988.0,0.0,population_1,пересчет от итогов ВПН-2002


In [16]:
%%time


cat_list = ['area_type',
            'age_group',
            'year_of_birth',
            'sex',
            'month',
            'birth_order',
            'mother_marital_status',
            'death_cause',
            'bride_age',
            'groom_age',
            'migration_direction',
            'country',
            'region',
            'territory_type',
            'registration_type',
            'citizenship']

super_nova_indicator_data = data_origin[[ 
                             'indicator_name',
                             'object_name',
                             'indicator_unit',
                             'original_unit',
                             'area_type',
                             'age_group',
                             'year_of_birth',
                             'sex',
                             'month',
                             'birth_order',
                             'mother_marital_status',
                             'death_cause',
                             'bride_age',
                             'groom_age',
                             'migration_direction',
                             'country',
                             'region',
                             'territory_type',
                             'registration_type',
                             'citizenship',
                             'original_accuracy',
                             'methodology'
                             ]]
super_nova_indicator_data = super_nova_indicator_data.drop_duplicates().sort_values(by = super_nova_indicator_data.columns.tolist()).reset_index(drop = True)

lst = []
for indicator in super_nova_indicator_data.indicator_name.unique():
    row = []
    row.append(indicator)
    for col in super_nova_indicator_data.columns[1:]:
        row.append(super_nova_indicator_data[super_nova_indicator_data.indicator_name == indicator][col].nunique())
    lst.append(row)

indicator_data = pd.DataFrame(lst, columns = super_nova_indicator_data.columns)    
def counter_cat(row):
    count = row['object_name']
    for col in cat_list:
        if row[col] != 0:
            count = count*row[col]
#         else:
#             count = count*1
    return count   

indicator_data['count'] = indicator_data.apply(counter_cat, axis = 1)
indicator_data['count'].sum()


CPU times: total: 5min 41s
Wall time: 5min 42s


705999093

In [17]:
for i in indicator_data.indicator_name.unique():
    display(indicator_data.query('indicator_name == @i'))

,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
0,Брутто-коэффициент воспроизводства населения,96,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,288


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
1,Возрастной коэффициент рождаемости,96,1,1,3,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2304


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
2,Естественный прирост,97,1,1,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,873


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
3,Коэффициент материнской смертности,98,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,294


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
4,Коэффициент миграционного прироста по субъектам,96,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,96


...

,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
96,Число умерших,107,1,1,3,4,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,6,3852


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
97,Число умерших в перинатальный период,106,1,1,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,5,954


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
98,Число умерших в ранний неонатальный период,106,1,1,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,5,954


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
99,Число умерших по месяцам,97,1,1,3,0,0,3,14,0,0,0,0,0,0,0,0,0,0,0,1,6,12222


,indicator_name,object_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,original_accuracy,methodology,count
100,Число умерших по причинам смерти,107,1,1,3,4,0,3,0,0,0,45,0,0,0,0,0,0,0,0,1,6,173340


In [18]:
print(data_origin.migration_direction.unique())
print(data_origin.migration_direction.nunique())

[nan 'Вся миграция' 'Внешняя для региона миграция'
 'Миграция в пределах России' 'Миграция в пределах России межрегиональная'
 'Миграция в пределах России внутрирегиональная' 'Миграция международная'
 'Миграция международная со странами СНГ'
 'Миграция международная с зарубежными странами, исключая страны СНГ'
 'Миграция международная со странами СНГ и Балтии'
 'Миграция международная с зарубежными странами, исключая страны СНГ и Балтии'
 'Миграция в пределах России с районами Крайнего Севера'
 'Миграция в пределах России межрегиональная с районами Крайнего Севера'
 'Миграция в пределах России внутрирегиональная с районами Крайнего Севера'
 'Миграция международная с другими зарубежными странами']
14


In [19]:
print(data_origin[cond_1].age_group.unique())
print(data_origin[cond_1].age_group.nunique())

['до 1 года' '1 год' '2 года' '3 года' '4 года' '5 лет' '6 лет' '7 лет'
 '8 лет' '9 лет' '10 лет' '11 лет' '12 лет' '13 лет' '14 лет' '15 лет'
 '16 лет' '17 лет' '18 лет' '19 лет' '20 лет' '21 год' '22 года' '23 года'
 '24 года' '25 лет' '26 лет' '27 лет' '28 лет' '29 лет' '30 лет' '31 год'
 '32 года' '33 года' '34 года' '35 лет' '36 лет' '37 лет' '38 лет'
 '39 лет' '40 лет' '41 год' '42 года' '43 года' '44 года' '45 лет'
 '46 лет' '47 лет' '48 лет' '49 лет' '50 лет' '51 год' '52 года' '53 года'
 '54 года' '55 лет' '56 лет' '57 лет' '58 лет' '59 лет' '60 лет' '61 год'
 '62 года' '63 года' '64 года' '65 лет' '66 лет' '67 лет' '68 лет'
 '69 лет' '70 лет' '71 год' '72 года' '73 года' '74 года' '75 лет'
 '76 лет' '77 лет' '78 лет' '79 лет' '80 лет' '81 год' '82 года' '83 года'
 '84 года' '85 лет' '86 лет' '87 лет' '88 лет' '89 лет' '90 лет' '91 год'
 '92 года' '93 года' '94 года' '95 лет' '96 лет' '97 лет' '98 лет'
 '99 лет' '100 лет и более' 'все возраста']
102


In [20]:
print(data_origin[cond_1 & cond_3].area_type.unique())  
print(data_origin[cond_1 & cond_3].area_type.nunique())  

['городское и сельское население' 'городское население'
 'сельское население']
3


In [21]:
print(data_origin[cond_1].age_group.unique())  
print(data_origin[cond_1].age_group.nunique())  

['до 1 года' '1 год' '2 года' '3 года' '4 года' '5 лет' '6 лет' '7 лет'
 '8 лет' '9 лет' '10 лет' '11 лет' '12 лет' '13 лет' '14 лет' '15 лет'
 '16 лет' '17 лет' '18 лет' '19 лет' '20 лет' '21 год' '22 года' '23 года'
 '24 года' '25 лет' '26 лет' '27 лет' '28 лет' '29 лет' '30 лет' '31 год'
 '32 года' '33 года' '34 года' '35 лет' '36 лет' '37 лет' '38 лет'
 '39 лет' '40 лет' '41 год' '42 года' '43 года' '44 года' '45 лет'
 '46 лет' '47 лет' '48 лет' '49 лет' '50 лет' '51 год' '52 года' '53 года'
 '54 года' '55 лет' '56 лет' '57 лет' '58 лет' '59 лет' '60 лет' '61 год'
 '62 года' '63 года' '64 года' '65 лет' '66 лет' '67 лет' '68 лет'
 '69 лет' '70 лет' '71 год' '72 года' '73 года' '74 года' '75 лет'
 '76 лет' '77 лет' '78 лет' '79 лет' '80 лет' '81 год' '82 года' '83 года'
 '84 года' '85 лет' '86 лет' '87 лет' '88 лет' '89 лет' '90 лет' '91 год'
 '92 года' '93 года' '94 года' '95 лет' '96 лет' '97 лет' '98 лет'
 '99 лет' '100 лет и более' 'все возраста']
102


In [22]:
print(data_origin[cond_1 & cond_3].sex.unique())  
print(data_origin[cond_1 & cond_3].sex.nunique())  

['женщины' 'мужчины' 'мужчины и женщины']
3


In [23]:
print(data_origin[cond_1].migration_direction.unique())  
print(data_origin[cond_1].migration_direction.nunique())  

[nan]
0


In [24]:
print(data_origin[cond_1 & cond_3].country.unique())  
print(data_origin[cond_1 & cond_3].country.nunique())  

[nan]
0


In [25]:
print(data_origin[cond_1 & cond_3].original_unit.unique())  
print(data_origin[cond_1 & cond_3].original_unit.nunique())  

['человек']
1


In [26]:
data_origin[cond_1 & cond_3 & (data_origin.country == 'Абхазия')]

,id,object_name,object_oktmo,object_okato,year,indicator_name,indicator_unit,original_unit,area_type,age_group,year_of_birth,sex,month,birth_order,mother_marital_status,death_cause,bride_age,groom_age,migration_direction,country,region,territory_type,registration_type,citizenship,indicator_value,original_accuracy,methodology,notes


In [27]:
for e in sorted(data_origin.age_group.unique().tolist()[1:]):

    print(e)

0 лет
0–14 лет
0–15 лет
0–17 лет
0–4 года
0–5 лет
1 год
10 лет
100 лет и более
10–14 лет
11 лет
12 лет
13 лет
14 лет
14 лет и более
14–29 лет
15 лет
15–17 лет
15–19 лет
15–49 лет
16 лет
17 лет
18 лет
18 лет и более
18–19 лет
18–44 года
19 лет
1–4 года
2 года
20 лет
20–24 года
20–34 года
21 год
22 года
23 года
24 года
25 лет
25–29 лет
26 лет
27 лет
28 лет
29 лет
3 года
30 лет
30–34 года
30–39 лет
31 год
32 года
33 года
34 года
35 лет
35 лет и более
35–39 лет
36 лет
37 лет
38 лет
39 лет
4 года
40 лет
40–44 года
40–49 лет
41 год
42 года
43 года
44 года
45 лет
45 лет и более
45–49 лет
46 лет
47 лет
48 лет
49 лет
5 лет
50 лет
50 лет и более
50–54 года
51 год
52 года
53 года
54 года
55 лет
55–59 лет
56 лет
57 лет
58 лет
59 лет
5–9 лет
6 лет
60 лет
60 лет и более
60–64 года
61 год
62 года
63 года
64 года
65 лет
65 лет и более
65–69 лет
66 лет
67 лет
68 лет
69 лет
6–13 лет
7 лет
70 лет
70–74 года
71 год
72 года
73 года
74 года
75 лет
75–79 лет
76 лет
77 лет
78 лет
79 лет
8 лет
80 лет
80 лет и 

In [28]:
for col in data_origin.columns:
    print(col + ' - ' + str((data_origin[col].isna().sum()/len(data_origin))*100)+'%')

id - 0.0%
object_name - 0.0%
object_oktmo - 24.984125899947188%
object_okato - 24.984125899947188%
year - 0.0%
indicator_name - 0.0%
indicator_unit - 0.0%
original_unit - 0.0%
area_type - 4.944561358281248%
age_group - 34.76320317766777%
year_of_birth - 86.8365249793725%
sex - 24.859338504758803%
month - 89.64478736949434%
birth_order - 97.14115246555205%
mother_marital_status - 99.79348414094167%
death_cause - 81.26581053406457%
bride_age - 94.04654068806715%
groom_age - 94.04654068806715%
migration_direction - 91.592438543332%
country - 86.1472180793554%
region - 95.23369105505837%
territory_type - 98.17546610864918%
registration_type - 99.54104018259042%
citizenship - 99.93833430025008%
indicator_value - 1.8523798088941426%
original_accuracy - 0.3512268423082799%
methodology - 0.15437836638782565%
notes - 65.8049563163823%


In [29]:
data_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340687 entries, 0 to 9340686
Data columns (total 28 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   object_name            object 
 2   object_oktmo           float64
 3   object_okato           float64
 4   year                   int64  
 5   indicator_name         object 
 6   indicator_unit         object 
 7   original_unit          object 
 8   area_type              object 
 9   age_group              object 
 10  year_of_birth          float64
 11  sex                    object 
 12  month                  object 
 13  birth_order            object 
 14  mother_marital_status  object 
 15  death_cause            object 
 16  bride_age              object 
 17  groom_age              object 
 18  migration_direction    object 
 19  country                object 
 20  region                 object 
 21  territory_type         object 
 22  registration_type 

In [30]:
data_origin.id.max()

9340687

In [31]:
data_object = data_origin[['object_name', 'object_oktmo', 'object_okato', 'year']]

In [32]:
data_object = data_object.drop_duplicates()

In [33]:
data_object = data_object[data_object.year > 2018]

In [34]:
data_object.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 668 to 8604655
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   object_name   395 non-null    object 
 1   object_oktmo  367 non-null    float64
 2   object_okato  367 non-null    float64
 3   year          395 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 15.4+ KB


In [35]:
data_object = data_object[['object_name', 'object_oktmo', 'object_okato']].drop_duplicates()

In [36]:
dict_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/!workspace/dict_ref_moduls/'
data_object.to_csv(dict_path + 'data_object.csv')

In [37]:
data_object.object_name.value_counts()

Бийск                                                     2
Барнаул                                                   2
Тамбов                                                    2
Липецк                                                    2
Елец                                                      2
Алтайский край                                            1
Новокузнецк                                               1
Новосибирск                                               1
Новороссийск                                              1
Новомосковск                                              1
Новокуйбышевск                                            1
Нижний Новгород                                           1
Нижний Тагил                                              1
Новочебоксарск                                            1
Нижнекамск                                                1
Нижневартовск                                             1
Нефтеюганск                             

In [38]:
data_origin.columns

Index(['id', 'object_name', 'object_oktmo', 'object_okato', 'year',
       'indicator_name', 'indicator_unit', 'original_unit', 'area_type',
       'age_group', 'year_of_birth', 'sex', 'month', 'birth_order',
       'mother_marital_status', 'death_cause', 'bride_age', 'groom_age',
       'migration_direction', 'country', 'region', 'territory_type',
       'registration_type', 'citizenship', 'indicator_value',
       'original_accuracy', 'methodology', 'notes'],
      dtype='object')

In [39]:
data_indicator = data_origin[['indicator_name', 'indicator_unit', 'original_unit', 'object_name', 'year',
       'area_type',
       'age_group', 'year_of_birth', 'sex', 'month', 'birth_order',
       'mother_marital_status', 'death_cause', 'bride_age', 'groom_age',
       'migration_direction', 'country', 'region', 'territory_type',
       'registration_type', 'citizenship',
       'original_accuracy', 'methodology', 'notes']].drop_duplicates()

In [40]:
dict_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/!workspace/dict_ref_moduls/'
data_indicator.to_csv(dict_path + 'data_indicator.csv')

In [41]:
data_indicator.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9340667 entries, 0 to 9340686
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   indicator_name         object 
 1   indicator_unit         object 
 2   original_unit          object 
 3   object_name            object 
 4   year                   int64  
 5   area_type              object 
 6   age_group              object 
 7   year_of_birth          float64
 8   sex                    object 
 9   month                  object 
 10  birth_order            object 
 11  mother_marital_status  object 
 12  death_cause            object 
 13  bride_age              object 
 14  groom_age              object 
 15  migration_direction    object 
 16  country                object 
 17  region                 object 
 18  territory_type         object 
 19  registration_type      object 
 20  citizenship            object 
 21  original_accuracy      float64
 22  methodology       

In [42]:
data_indicator[['indicator_name', 'indicator_unit', 'original_unit']].drop_duplicates().sort_values(by='indicator_name')

,indicator_name,indicator_unit,original_unit
3610668,Брутто-коэффициент воспроизводства населения,промилле,промилле
3532908,Возрастной коэффициент рождаемости,число родившихся детей на 1000 женщин данного ...,число родившихся детей на 1000 женщин данного ...
2778930,Естественный прирост,человек,человек
5512341,Коэффициент материнской смертности,число материнских смертей на 100000 рожденных ...,число материнских смертей на 100000 рожденных ...
8598601,Коэффициент миграционного прироста по субъектам,на 10000 человек,на 10000 человек
5807127,Коэффициент младенческой смертности,число смертей на 1000 родившихся живыми,число смертей на 1000 родившихся живыми
2821056,Коэффициент общего прироста населения,промилле,промилле
5587440,Коэффициент перинатальной смертности,промилле,промилле
5554245,Коэффициент ранней неонатальной смертности,промилле,промилле
5471301,Материнская смертность,человек,человек


In [43]:
for u in data_origin.citizenship.unique():
    print(u)

nan
Российское гражданство
иностранное гражданство
без гражданства
всего


In [44]:
data_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340687 entries, 0 to 9340686
Data columns (total 28 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   object_name            object 
 2   object_oktmo           float64
 3   object_okato           float64
 4   year                   int64  
 5   indicator_name         object 
 6   indicator_unit         object 
 7   original_unit          object 
 8   area_type              object 
 9   age_group              object 
 10  year_of_birth          float64
 11  sex                    object 
 12  month                  object 
 13  birth_order            object 
 14  mother_marital_status  object 
 15  death_cause            object 
 16  bride_age              object 
 17  groom_age              object 
 18  migration_direction    object 
 19  country                object 
 20  region                 object 
 21  territory_type         object 
 22  registration_type 

In [45]:
data_origin_methodology = pd.read_csv(data_origin_path + 'methodology.csv')
data_origin_structure = pd.read_csv(data_origin_path + 'structure-20210430.csv')

In [46]:
data_origin_structure

,field name,english description,russian description,format
0,id,Observation ID within the dataset,ID наблюдения в рамках набора данных,numeric
1,object_name,Name of an object of observation,Название объекта наблюдения,string
2,object_oktmo,The OKTMO code of an object,Код ОКТМО объекта наблюдения,string
3,object_okato,The OKATO code of an object,Код ОКАТО объекта наблюдения,string
4,year,Year of observation,Год наблюдения,numeric
5,indicator_name,Indicator name,Название показателя,string
6,indicator_unit,"Indicator unit, as given in the dataset","Единица измерения показателя для значений, при...",string
7,original_unit,"Indicator unit, as given in the original data ...",Единица измерения показателя в источнике данных,string
8,area_type,Area type,Тип поселения,string
9,age_group,Age group,Возрастная группа,string


In [47]:
data_origin_structure[data_origin_structure['field name'] == 'citizenship']['russian description'].values

array(['Гражданство'], dtype=object)

In [48]:
data_origin.columns

Index(['id', 'object_name', 'object_oktmo', 'object_okato', 'year',
       'indicator_name', 'indicator_unit', 'original_unit', 'area_type',
       'age_group', 'year_of_birth', 'sex', 'month', 'birth_order',
       'mother_marital_status', 'death_cause', 'bride_age', 'groom_age',
       'migration_direction', 'country', 'region', 'territory_type',
       'registration_type', 'citizenship', 'indicator_value',
       'original_accuracy', 'methodology', 'notes'],
      dtype='object')

In [49]:
listik = []
for u in data_origin.indicator_name.unique():
    for w in u.split('по ')[1:]:
        listik.append(w)

for x in set(listik):
    print(x)

городам — число прибывших
возрастным группам
месяцам
видам и срокам регистрации — число прибывших
полу и возрасту в районах Крайнего Севера и местностях, приравненных к ним — число выбывших
городам — миграционный прирост
брачному состоянию матери
видам и срокам регистрации — число выбывших
полу и возрасту в районах Крайнего Севера и местностях, приравненных к ним — миграционный прирост
возрасту матери и очередности рождения
городам — число выбывших
возрасту жениха и невесты
полу и возрасту — число прибывших
районам Крайнего Севера и местностям, приравненных к ним — число выбывших
полу и возрасту — число выбывших
очередности рождения
видам и срокам регистрации — миграционный прирост
субъектам — миграционный прирост
районам Крайнего Севера и местностям, приравненных к ним — миграционный прирост
полу и возрасту в районах Крайнего Севера и местностях, приравненных к ним — число прибывших
субъектам — число выбывших
полу и возрасту — миграционный прирост
причинам смерти
полу и возрасту 
райо

In [50]:
col_dict = {
    'object_name':'Субъекты',
    'area_type':'Тип поселения',
    'age_group':'Возрастная группа',
    'year_of_birth':'Год рождения',
    'sex':'Пол',
    'month':'Месяц',
    'birth_order':'Очередность рождения',
    'mother_marital_status':'Брачное состояние матери',
    'death_cause':'Причина смерти',
    'bride_age':'Возрастная группа невесты',
    'groom_age':'Возрастная группа жениха',
    'migration_direction':'Направление потока миграции',
    'country':'Страна миграции (для прибывших — откуда прибыли или последнее место жительства; для выбывших — куда выбыли или новое место жительства)',
    'region':'Регион миграции (для прибывших — откуда прибыли или последнее место жительства; для выбывших — куда выбыли или новое место жительства)',
    'territory_type':'Тип территории в границах региона миграции (районы Крайнего Севера или другие территории)',
    'registration_type':'Вид и срок регистрации',
    'citizenship':'Гражданство'
}

def guidebook(data):
    doc = docx.Document()
    i = 0
    ind = []
    for u in sorted(data.indicator_name.unique()):
        i += 1
        data_cirle = data[data.indicator_name == u]
        # if len(data_cirle.object_name.unique()) > 1 and 'по субъектам' not in u:
        #     sub_part = ' по субъектам'
        # else:
        #     sub_part = ''


        print('Показатель номер ' + str(i) + ': ' + u)
        print('Число строк данных для показателя: ' + str(len(data_cirle)))
        original_unit_list = data_cirle.original_unit.unique()
        indicator_unit_list = data_cirle.indicator_unit.unique()
  

        print('Единица измерения показателя: ')
        if len(original_unit_list) == 1 and original_unit_list[0] == indicator_unit_list[0]:
            print(original_unit_list[0])
        elif len(original_unit_list) != 1:
            ind.append(i)
            for n in range(len(original_unit_list)):
                print('Для показателя в период ' + 
                      str(data_cirle[data_cirle.original_unit == original_unit_list[n]]['year'].min()) +
                      ' - ' +
                      str(data_cirle[data_cirle.original_unit == original_unit_list[n]]['year'].max()) + ' - ' + 
                      str(original_unit_list[n]))
        print('Атрибуты, по которым определяется показатель:')
        for k in col_dict:
            if len(data_cirle[k].unique()) > 1:
                print(' - ' + col_dict[k] + ':')
                print(list(data_cirle[k].unique()))

        # Начало создания вордовского документа        
        doc.add_page_break()

        # Заголовок и пустая строка
        doc.add_heading((str(i) + ' - ' + u), 1)
        doc.add_paragraph()

        # Абзац с размером датасета для показателя
        para = doc.add_paragraph()
        bold_para = para.add_run('Число строк данных для показателя: \n')  
        bold_para.bold = True
        para.add_run(str(len(data_cirle)))

        # Абзац с единицей измерения
        para = doc.add_paragraph()
        bold_para = para.add_run('Единица измерения показателя: \n')  
        if len(original_unit_list) == 1 and original_unit_list[0] == indicator_unit_list[0]:
            bold_para.bold = True
            para.add_run(original_unit_list[0])
        elif len(original_unit_list) != 1:

            for n in range(len(original_unit_list)):
                text = ('Для показателя в период ' + 
                      str(data_cirle[data_cirle.original_unit == original_unit_list[n]]['year'].min()) +
                      ' - ' +
                      str(data_cirle[data_cirle.original_unit == original_unit_list[n]]['year'].max()) + ' - ' + 
                      str(original_unit_list[n]) + '\n')
                bold_para.bold = True
                para.add_run(text)        
        
        # Абзац с перечнем атрибутов
        para = doc.add_paragraph()
        bold_para = para.add_run('Атрибуты, по которым определяется показатель: \n')  
        bold_para.bold = True
        for k in col_dict:
            if len(data_cirle[k].unique()) > 1:
                underline_para = para.add_run((col_dict[k] + ': \n'))
                underline_para.underline = True
                underline_para.bold = True
                para.add_run((str(list(data_cirle[k].unique())) + '\n')) 



        
    doc.save('test.docx')


    # print('Номера показателей с несколькими единицами измерения:')
    # print(ind)


    

In [51]:
guidebook(data_origin)

Показатель номер 1: Брутто-коэффициент воспроизводства населения
Число строк данных для показателя: 8640
Единица измерения показателя: 
промилле
Атрибуты, по которым определяется показатель:
 - Субъекты:
['Алтайский край', 'Амурская область', 'Архангельская область', 'Архангельская область без Ненецкого автономного округа', 'Астраханская область', 'Белгородская область', 'Брянская область', 'Владимирская область', 'Волгоградская область', 'Вологодская область', 'Воронежская область', 'Дальневосточный федеральный округ', 'Еврейская автономная область', 'Забайкальский край', 'Ивановская область', 'Иркутская область', 'Кабардино-Балкарская Республика', 'Калининградская область', 'Калужская область', 'Камчатский край', 'Карачаево-Черкесская Республика', 'Кемеровская область – Кузбасс', 'Кировская область', 'Костромская область', 'Краснодарский край', 'Красноярский край', 'Курганская область', 'Курская область', 'Ленинградская область', 'Липецкая область', 'Магаданская область', 'Москва', '

In [52]:
data_origin.year.min()

1990

In [53]:
data_potato = pd.read_csv(r"C:\Users\User\Desktop\Росстат\Проект_ИИ\Картофель.csv", sep='#')

In [54]:
max_len = 0
for i, row in data_potato.iterrows():
    if max_len < len(row['original_name']):
        max_len = len(row['original_name'])
        id = i

print(max_len)
print(data_potato.loc[id, 'original_name'])

309
00132180 Блокнот А6 40л.клCoffee Bookгреб.обл.карт. Хатбер 1268570000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [55]:
data_origin[(data_origin.indicator_name == "Миграция по видам и срокам регистрации — число прибывших") & 
            (data_origin.year.isin([2013,2014])) & 
            (data_origin.region.isin(['Москва', 'Санкт-Петербург'])) & 
            (data_origin.registration_type == "прибыли и зарегистрированы в новом месте жительства")][['object_name', 'year', 'indicator_name', 'indicator_unit', 'region', 'registration_type', 'indicator_value']]

,object_name,year,indicator_name,indicator_unit,region,registration_type,indicator_value
9306268,Российская Федерация,2013,Миграция по видам и срокам регистрации — число...,человек,Москва,прибыли и зарегистрированы в новом месте жител...,47413.0
9306269,Российская Федерация,2014,Миграция по видам и срокам регистрации — число...,человек,Москва,прибыли и зарегистрированы в новом месте жител...,47413.0
9309508,Российская Федерация,2013,Миграция по видам и срокам регистрации — число...,человек,Санкт-Петербург,прибыли и зарегистрированы в новом месте жител...,66816.0
9309509,Российская Федерация,2014,Миграция по видам и срокам регистрации — число...,человек,Санкт-Петербург,прибыли и зарегистрированы в новом месте жител...,66816.0


In [56]:
end = time.time()
duration = end-start
print(int(duration), 'сек. или', int(duration//60), 'мин. и ', int(duration % 60), 'сек.')

810 сек. или 13 мин. и  30 сек.
